<a href="https://colab.research.google.com/github/AndresPlazas19931504/PROYECTOINTEGRADOR-PREICA2501B020071/blob/main/S25_Evidencia_de_Aprendizaje_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importación de dataset y analisis detallado de su estructura

In [1]:
# Importamos las bibliotecas que vamos a usar.
from IPython import get_ipython
from IPython.display import display
# Para manejar archivos.
import io
# Para manejar tablas de datos (DataFrames).
import pandas as pd
# Para acceder a datasets de Kaggle.
import kagglehub
# Para adaptar los datasets de Kaggle.
from kagglehub import KaggleDatasetAdapter
# Para mostrar datos en el libro.
from IPython.display import display

# Definimos la ruta del archivo que queremos cargar.
file_path = "2019-Oct.csv"

# Cargamos el dataset de Kaggle usando 'kagglehub'.
df = kagglehub.load_dataset(
  # Usamos pandas para leer el dataset.
  KaggleDatasetAdapter.PANDAS,
  # Nombre del dataset en Kaggle.
  "mkechinov/ecommerce-behavior-data-from-multi-category-store",
  # Ruta del archivo dentro del dataset.
  file_path,
)

<ipython-input-1-b979563c9fc9>:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 1.61G/1.61G [00:18<00:00, 95.6MB/s]

Extracting zip of 2019-Oct.csv...


In [2]:
# Mostramos las primeras 5 filas del DataFrame para ver cómo son los datos.
display(df.head())

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [3]:
# Obtenemos información general del DataFrame.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [4]:
# Obtenemos estadísticas descriptivas del DataFrame.
df.describe()

,product_id,category_id,price,user_id
count,4.244876e+07,4.244876e+07,4.244876e+07,4.244876e+07
mean,1.054993e+07,2.057404e+18,2.903237e+02,5.335371e+08
std,1.188191e+07,1.843926e+16,3.582692e+02,1.852374e+07
min,1.000978e+06,2.053014e+18,0.000000e+00,3.386938e+07
25%,1.005157e+06,2.053014e+18,6.598000e+01,5.159043e+08
50%,5.000470e+06,2.053014e+18,1.629300e+02,5.296965e+08
75%,1.600030e+07,2.053014e+18,3.585700e+02,5.515788e+08
max,6.050001e+07,2.175420e+18,2.574070e+03,5.662809e+08


# Exploramos las columnas validando los valores únicos en la columna y contamos cuántas veces aparece cada valor.


In [5]:
df['event_time'].unique()
df['event_time'].value_counts()

,count
event_time,
2019-10-15 08:52:00 UTC,116
2019-10-13 09:44:03 UTC,114
2019-10-24 17:07:55 UTC,113
2019-10-13 11:15:43 UTC,111
2019-10-26 09:51:31 UTC,108
...,...
2019-10-30 21:55:19 UTC,1
2019-10-21 23:03:58 UTC,1
2019-10-21 23:03:55 UTC,1


In [6]:
df['event_type'].unique()
df['event_type'].value_counts()

,count
event_type,
view,40779399
cart,926516
purchase,742849


In [7]:
df['product_id'].unique()
df['product_id'].value_counts()

,count
product_id,
1004856,500354
1004767,438232
1005115,355786
1004833,237545
1004249,231070
...,...
58100161,1
58100153,1
58100160,1


In [8]:
df['category_id'].unique()
df['category_id'].value_counts()

,count
category_id,
2053013555631882655,11507231
2053013553559896355,1888744
2053013558920217191,1130645
2053013554415534427,1105635
2053013554658804075,1100188
...,...
2053013555690602915,4
2053013560388223675,3
2053013559381590653,2


In [9]:
df['category_code'].unique()
df['category_code'].value_counts()

,count
category_code,
electronics.smartphone,11507231
electronics.clocks,1311033
computers.notebook,1137623
electronics.video.tv,1113750
electronics.audio.headphone,1100188
...,...
apparel.shorts,632
construction.tools.soldering,632
apparel.jacket,436


In [10]:
df['brand'].unique()
df['brand'].value_counts()

,count
brand,
samsung,5282775
apple,4122554
xiaomi,3083763
huawei,1111205
lucente,655861
...,...
skindoctors,1
ecolaboratorie,1
sx,1


In [11]:
df['price'].unique()
df['price'].value_counts()

,count
price,
257.15,224869
51.22,144965
231.64,122738
131.53,113518
51.46,111273
...,...
364.30,1
168.63,1
422.53,1


In [12]:
df['user_id'].unique()
df['user_id'].value_counts()

,count
user_id,
512475445,7436
512365995,4013
526731152,2912
512505687,2894
513021392,2862
...,...
532704496,1
555677537,1
566280351,1


In [13]:
df['user_session'].unique()
df['user_session'].value_counts()

,count
user_session,
fb075266-182d-4c11-b5f7-4e4dcdabd4a7,1159
cfb90a35-9575-495c-b6aa-48ddca2a7a9c,1137
2183f046-46f1-4ff6-96ef-f74986e7c8a1,584
b2101293-44c1-4814-836a-94b0c03bb9c2,564
e9d2b8ad-3e69-47f2-8b8a-1fadfc583756,425
...,...
d8a4a956-1d7a-4ae8-95f1-605402901cef,1
0b12f578-8cfd-4a84-a63f-6a2f0c3d421c,1
f055b83d-02a3-43ca-bc16-2128c26c671b,1


# Instalamos bibliotecas adicionales.

In [ ]:
!pip install kagglehub[pandas-datasets]
!pip install --upgrade ydata-profiling
!pip install --no-cache-dir ydata-profiling

In [ ]:
# Importamos las bibliotecas que acabamos de instalar.
# Para acceder a datasets de Kaggle.
import kagglehub
# Para adaptar los datasets de Kaggle.
from kagglehub import KaggleDatasetAdapter
# Para manejar tablas de datos (DataFrames).
import pandas as pd
# Para generar reportes de análisis de datos.
import ydata_profiling as pp
# Para crear reportes de perfiles de datos.
from ydata_profiling import ProfileReport

# Definimos la ruta del archivo que queremos cargar.
file_path = "2019-Oct.csv"

# Cargamos el dataset de Kaggle usando 'kagglehub'.
df = kagglehub.load_dataset(
  # Usamos pandas para leer el dataset.
  KaggleDatasetAdapter.PANDAS,
  # Nombre del dataset en Kaggle.
  "mkechinov/ecommerce-behavior-data-from-multi-category-store",
  # Ruta del archivo dentro del dataset.
  file_path,
)

# Tomamos una muestra del DataFrame para el reporte.
# Tomamos el 12.5% del dataset.
df_muestra = df.sample(frac=0.125)


# Creamos un reporte de análisis de datos con 'ydata-profiling'.
# Creamos el reporte.
reporte = ProfileReport(df_muestra, minimal=False)
# Guardamos el reporte en un archivo HTML.
reporte.to_file(output_file='reporte.html')